In [4]:
# -*- coding: utf-8 -*-
"""
LOOK-AHEAD INTEGRATION + DYNAMIC VRPP TRACE + VISUALIZATION
Héctor Bonilla - Full integration with Risk-based Lookahead (corrected MUST-GO logic)
Requirements: pandas, gurobipy, matplotlib, openpyxl
Adjust file paths before running.
"""

# --------------------------
# Import libraries
import time

import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# --------------------------
# FILE PATHS (adjust to your environment)
file_parameters = r"../data/wsr_simulator/daily_waste/parameters.xlsx"
file_distances = r"../data/wsr_simulator/distance_matrix/distance_104bins.xlsx"

# Output Excel files for KPIs, flows, dynamic traces, overflow risk, and lookahead traces
output_excel_kpi = r"../assets/output/31_days/riomaior_104/TestVRPP-104bins_KPI_DYNAMIC_LOOKAHEAD_CORRECTED.xlsx"
output_excel_flows = r"../assets/output/31_days/riomaior_104/TestVRPP-104bins_Flows_LOOKAHEAD_CORRECTED.xlsx"
output_excel_dynamic = output_excel_kpi.replace("_KPI_", "_DYNAMIC_")
output_excel_risk = output_excel_kpi.replace("_KPI_", "_OVERFLOW_RISK_TRACE_")
output_excel_lookahead = output_excel_kpi.replace("_KPI_", "_LOOKAHEAD_TRACE_")

# --------------------------
# PARAMETERS (adjust to model settings)
C = 1.0  # Cost per unit distance
R = 0.5837  # Revenue per kg collected
Omega = 0.1  # Vehicle fixed cost per route
Q = 3500.0  # Vehicle flow capacity per arc (kg)
delta = 0  # Tolerance in "hit" constraint
T_days = 30  # Total simulation days
T = list(range(1, T_days + 1))
H_max = 30  # Maximum look-ahead horizon in days
B = 19  # Parameter for conversion from percentage to kg
vol = 2.5  # Volume factor

# --------------------------
# READ INPUTS FROM EXCEL
# df_params: initial fill (Si), capacity (Ei), mean accumulation per day
df_params = pd.read_excel(file_parameters, sheet_name="Si_Ei", engine="openpyxl")
IDs_model = df_params["ID"].astype(int).tolist()  # List of bin IDs
S_base = {
    int(r["ID"]): float(r["Si"]) for _, r in df_params.iterrows()
}  # Initial fill in kg
E_dict = {
    int(r["ID"]): float(r["Ei"]) for _, r in df_params.iterrows()
}  # Overflow threshold (capacity)
avg_ai = {
    int(r["ID"]): float(r["Mean"]) for _, r in df_params.iterrows()
}  # Mean daily accumulation in kg/day

# df_ait: daily accumulation per bin as percentage, convert to kg
df_ait = pd.read_excel(file_parameters, sheet_name="%ai_t", engine="openpyxl")
ait_kg_dict = {}
for _, row in df_ait.iterrows():
    i = int(row["ID"])
    for t in T:
        col = f"Day {t}"
        # convert percentage to kg: (percent/100) * B * vol
        if col in df_ait.columns and not pd.isna(row[col]):
            ait_kg_dict[(i, t)] = float(row[col]) / 100.0 * B * vol
        else:
            ait_kg_dict[(i, t)] = 0.0  # if no data, assume 0 accumulation

# --------------------------
# DISTANCE MATRIX
df_dist = pd.read_excel(file_distances, engine="openpyxl", index_col=0)
df_dist.index = df_dist.index.astype(int)
df_dist.columns = df_dist.columns.astype(int)

nodes_real = IDs_model  # Real bin nodes
n_bins = len(nodes_real)  # Number of bins
fictitious_node = n_bins + 1  # Fictitious node to handle VRP end
nodes = [0] + nodes_real + [fictitious_node]  # All nodes: 0=depot, fictitious_node=sink

# Build distance dictionary (include mapping for fictitious node)
distances = {}
for i in nodes:
    for j in nodes:
        ii = 0 if i == fictitious_node else i
        jj = 0 if j == fictitious_node else j
        distances[(i, j)] = float(df_dist.loc[ii, jj])


# --------------------------
# HELPER FUNCTIONS
def will_cause_overflow_nextday(S_current_i, avg_ai_i, E_i):
    """
    Predict whether a bin will overflow tomorrow based on current fill and average accumulation.
    If S_current + 2*avg_ai >= Ei -> risk of overflow, route should be planned today.
    """
    return (S_current_i + 2.0 * avg_ai_i) >= E_i


def compute_nc_for_overflow_bin(i, t, avg_ai, E_dict, T_max):
    """
    Compute next collection day (nc) for bin i.
    Start from t+1, sum daily avg_ai until predicted fill exceeds Ei.
    If no overflow within horizon, return T_max+1.
    """
    S_temp = 0.0
    for tt in range(t + 1, T_max + 1):
        S_temp += avg_ai.get(i, 0.0)
        if S_temp >= E_dict[i]:
            return tt
    return T_max + 1


def will_overflow_between_t_and_nc(i, t, nc, S_current, avg_ai, E_dict):
    """
    Check if bin i will overflow at any day in [t, nc], using avg_ai projections.
    """
    S_temp = S_current[i]
    for tt in range(t, nc + 1):
        if tt > t:
            S_temp += avg_ai.get(i, 0.0)
        if S_temp >= E_dict[i]:
            return True
    return False


# --------------------------
# INITIAL STATE
S_current = {i: S_base[i] for i in nodes_real}  # current fill (kg) at start

# OUTPUT TABLES
dynamic_rows = []  # per-bin daily dynamics
kpi_rows = []  # daily KPIs
flow_rows = []  # arcs and collection variables
overflow_risk_rows = []  # trace of S_current + 2*avg_ai
lookahead_rows = []  # lookahead predicted fills per bin
mg_trace_rows = []  # MUST-GO membership trace

# --------------------------
# MAIN SIMULATION LOOP
for t in T:
    print(f"\n=== Day {t} ===")
    t_start_time = time.time()

    # Compute Wit_before: fill before collection plus today's accumulation
    Wit_before_dict = {
        i: S_current[i] + ait_kg_dict.get((i, t), 0.0) for i in nodes_real
    }

    # Step A: determine whether a route is needed today (overflow trigger)
    overflow_today_bins = []
    route_needed_today = False
    for i in nodes_real:
        if will_cause_overflow_nextday(S_current[i], avg_ai.get(i, 0.0), E_dict[i]):
            overflow_today_bins.append(i)
            route_needed_today = True

    print(
        "Initial trigger - bins predicted to overflow (S+2*avg >= Ei):",
        overflow_today_bins,
    )

    # Record overflow risk for each bin (trace table)
    for i in nodes_real:
        risk_val = S_current[i] + 2.0 * avg_ai.get(i, 0.0)
        overflow_risk_rows.append(
            {
                "Day": t,
                "ID": i,
                "S_current": S_current[i],
                "2x_avg_ai": 2.0 * avg_ai.get(i, 0.0),
                "S_current+2avg": risk_val,
                "Ei": E_dict[i],
                "Margin_to_Ei": E_dict[i] - risk_val,
                "RiskStatus": "RISK" if risk_val >= E_dict[i] else "SAFE",
            }
        )

    # If no route needed today, accumulate and continue
    if not route_needed_today:
        print(
            "No route needed today (no triggered overflow). Accumulating a_it for all bins."
        )
        for i in nodes_real:
            S_before = S_current[i]
            a_it = ait_kg_dict.get((i, t), 0.0)
            Wit_before = S_before + a_it
            collected_planned = 0
            collected_actual = 0
            Wit_collected = 0.0
            Profit_before = R * Wit_before if collected_planned == 1 else 0.0
            Profit_after = 0.0
            S_after = Wit_before
            dynamic_rows.append(
                {
                    "Day": t,
                    "ID": i,
                    "S_before": S_before,
                    "a_it": a_it,
                    "Wit_before": Wit_before,
                    "Collected_planned": collected_planned,
                    "Collected_actual": collected_actual,
                    "Wit_collected": Wit_collected,
                    "Wit_after": S_after,
                    "Profit_before": Profit_before,
                    "Profit_after": Profit_after,
                    "RiskStatus": "SAFE" if Wit_before < E_dict[i] else "RISK",
                }
            )
            S_current[i] = S_after
        # Append KPI row with zero collections
        kpi_rows.append(
            {
                "Day": t,
                "KgCollected": 0.0,
                "KgOverflow": 0.0,
                "NumBinsOverflow_All": 0,
                "IDsBinsOverflow_All": [],
                "Distance": 0.0,
                "Profit": 0.0,
                "VehicleCost": 0.0,
                "FO": 0.0,
                "NumVehicles": 0,
                "BinsVisited": 0,
                "IDsBinsVisited": [],
                "BinsNotVisited": len(nodes_real),
                "IDsBinsNotVisited": nodes_real,
                "Routes": [],
                "SolveTime_s": 0.0,
            }
        )
        continue

    # Step B: compute nc (next collection day) based on overflow bins
    T_max = min(max(T), t + H_max)  # cap horizon
    candidate_nc_days = []
    for i in overflow_today_bins:
        nc_i = compute_nc_for_overflow_bin(i, t, avg_ai, E_dict, T_max)
        if nc_i <= T_max:
            candidate_nc_days.append(nc_i)
    nc = min(candidate_nc_days) if candidate_nc_days else t
    print(f"Computed next collection day nc = {nc} (based on overflow_today_bins)")

    # Step C: define MUST-GO set MG = bins expected to overflow between t and nc
    planned_bins = []
    for i in nodes_real:
        if will_overflow_between_t_and_nc(i, t, nc, S_current, avg_ai, E_dict):
            planned_bins.append(i)
    planned_bins = sorted(set(planned_bins))
    collected_planned_dict = {i: 1 if i in planned_bins else 0 for i in nodes_real}
    print("MUST-GO set (planned_bins MG) between t and nc:", planned_bins)

    # Save lookahead trace for visualization
    for i in nodes_real:
        S_temp = S_current[i]
        for tt in range(t, nc + 1):
            if tt > t:
                S_temp += avg_ai.get(i, 0.0)
            lookahead_rows.append(
                {
                    "Day": t,
                    "ID": i,
                    "HorizonDay": tt,
                    "PredictedFill": S_temp,
                    "Ei": E_dict[i],
                    "DecisionLookahead": 1 if i in planned_bins else 0,
                    "S_current+2avg_ai": S_current[i] + 2.0 * avg_ai.get(i, 0.0),
                    "nc": nc,
                }
            )
        mg_trace_rows.append(
            {
                "Day": t,
                "ID": i,
                "In_MG": 1 if i in planned_bins else 0,
                "nc": nc,
                "S_current": S_current[i],
            }
        )

    # --------------------------
    # BUILD VRP MODEL (Gurobi)
    model = gp.Model(f"VRP_Lookahead_Day_{t}")
    x = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")  # Arc usage
    g = model.addVars(nodes_real, vtype=GRB.BINARY, name="g")  # Whether bin visited
    f = model.addVars(
        nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f"
    )  # Flow in kg

    # Objective: Revenue from collected kg minus distance cost minus vehicle fixed cost
    model.setObjective(
        R * gp.quicksum(Wit_before_dict[i] * g[i] for i in nodes_real)
        - C
        * gp.quicksum(
            x[i, j] * distances[(i, j)] for i in nodes for j in nodes if i != j
        )
        - Omega * gp.quicksum(x[0, j] for j in nodes_real),
        GRB.MAXIMIZE,
    )

    # 1) If Wit_before==0, cannot collect
    model.addConstrs(
        (g[i] == 0 for i in nodes_real if Wit_before_dict.get(i, 0.0) == 0.0),
        name="no_collect_empty",
    )
    # 2) Enforce MUST-GO bins
    for i in planned_bins:
        model.addConstr(g[i] == 1)
    # 3) Linking arcs: sum incoming arcs == g[j] for each bin
    model.addConstrs(
        (
            gp.quicksum(x[i, j] for i in nodes if i != fictitious_node and i != j)
            == g[j]
            for j in nodes_real
        ),
        name="linking_in",
    )
    # 4) g bounds <=1
    model.addConstrs((g[i] <= 1 for i in nodes_real), name="g_ub1")
    # 5) Hit constraint: serve at least planned MG bins
    model.addConstr(
        gp.quicksum(g[i] for i in planned_bins) >= len(planned_bins) - n_bins * delta,
        name="hit_constraint",
    )
    # 6) Depot balance: leaving depot == returning
    model.addConstr(
        gp.quicksum(x[0, j] for j in nodes if j != 0)
        - gp.quicksum(x[i, 0] for i in nodes if i != fictitious_node)
        == 0,
        name="depot_balance",
    )
    # 7) Node balance for intermediate nodes
    model.addConstrs(
        (
            gp.quicksum(x[i, h] for i in nodes if i != fictitious_node)
            - gp.quicksum(x[h, j] for j in nodes if j != 0)
            == 0
            for h in nodes_real
        ),
        name="node_balance",
    )
    # 8) Flow capacity linking: f <= Q*x
    model.addConstrs(
        (f[i, j] <= Q * x[i, j] for i in nodes for j in nodes if i != j),
        name="flow_capacity",
    )
    # 9) Flow balance: collected mass
    model.addConstrs(
        (
            gp.quicksum(f[i, j] for j in nodes if j != 0)
            - gp.quicksum(f[j, i] for j in nodes if j != fictitious_node)
            == Wit_before_dict[i] * g[i]
            for i in nodes_real
        ),
        name="flow_balance_mass",
    )

    # Solver parameters
    model.Params.TimeLimit = 300
    model.Params.MIPGap = 1e-3
    model.optimize()

    # --------------------------
    # UPDATE DYNAMICS PER BIN
    print("\nDay summary (per bin):")
    total_collected_kg = 0.0
    for i in nodes_real:
        S_before = S_current[i]
        a_it = ait_kg_dict.get((i, t), 0.0)
        Wit_before = Wit_before_dict[i]
        collected_planned = collected_planned_dict[i]
        collected_actual = 1 if (hasattr(g[i], "X") and g[i].X > 0.5) else 0
        Wit_collected = Wit_before if collected_actual == 1 else 0.0
        Profit_before = R * Wit_before if collected_planned == 1 else 0.0
        Profit_after = R * Wit_collected
        total_collected_kg += Wit_collected
        S_after = 0.0 if collected_actual == 1 else (S_before + a_it)

        dynamic_rows.append(
            {
                "Day": t,
                "ID": i,
                "S_before": S_before,
                "a_it": a_it,
                "Wit_before": Wit_before,
                "Collected_planned": collected_planned,
                "Collected_actual": collected_actual,
                "Wit_collected": Wit_collected,
                "Wit_after": S_after,
                "Profit_before": Profit_before,
                "Profit_after": Profit_after,
                "In_MG": 1 if i in planned_bins else 0,
                "nc": nc,
            }
        )

        print(
            f"Bin {i} | S_before={S_before:.2f} | a_it={a_it:.2f} | Wit_before={Wit_before:.2f} | "
            f"Planned={collected_planned} | Actual={collected_actual} | Wit_collected={Wit_collected:.2f} | "
            f"Wit_after={S_after:.2f} | Profit_before={Profit_before:.2f} | Profit_after={Profit_after:.2f}"
        )

        # update state
        S_current[i] = S_after

# --------------------------
# EXPORT RESULTS
pd.DataFrame(dynamic_rows).to_excel(output_excel_dynamic, index=False)
pd.DataFrame(kpi_rows).to_excel(output_excel_kpi, index=False)
pd.DataFrame(flow_rows).to_excel(output_excel_flows, index=False)
pd.DataFrame(overflow_risk_rows).to_excel(output_excel_risk, index=False)
pd.DataFrame(lookahead_rows).to_excel(output_excel_lookahead, index=False)

print(
    "\nSimulation completed. All dynamic, KPI, flow, overflow risk, and lookahead tables exported."
)


=== Day 1 ===
Initial trigger - bins predicted to overflow (S+2*avg >= Ei): [1173, 1655, 6092, 11571]
Computed next collection day nc = 10 (based on overflow_today_bins)
MUST-GO set (planned_bins MG) between t and nc: [197, 787, 892, 1171, 1172, 1173, 1410, 1459, 1606, 1621, 1655, 3224, 3315, 5752, 5931, 5933, 5938, 6046, 6092, 6225, 6240, 6904, 6911, 7245, 7445, 7733, 10799, 11213, 11571, 11781, 11789, 11790, 11800, 11811, 11824, 11860, 11873, 11983, 11986, 11989, 12036, 12344, 13138, 13579, 13633, 16825, 17812, 18141]
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 25.10")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900KS, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 11601 rows, 22576 columns and 76958 nonzeros
Model fingerprint: 0x27962904
Variable types: 11236 continuous, 11340 integer (11340 binary)
C

In [5]:
# -*- coding: utf-8 -*-
"""
DYNAMIC VRPP VISUALIZATION WITH FULL KPI TOOLTIP
Héctor Bonilla - Interactive Dashboard with Look-Ahead, Overflow, MG, and KPIs
Requirements: pandas, plotly, openpyxl
"""

import pandas as pd
import plotly.graph_objects as go

# --------------------------
# FILE PATHS
file_coordinates = r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-Determinsit_104bins\coordinates104.xlsx"
file_kpi = output_excel_kpi  # exportado previamente por la simulación
file_flows = output_excel_flows  # exportado previamente por la simulación

# --------------------------
# READ DATA
# Leer coordenadas directamente desde el archivo
df_coords = pd.read_excel(file_coordinates, engine="openpyxl")

# Crear diccionarios de coordenadas usando ID como clave
# X = Lat, Y = Lon (puedes cambiar si tu sistema de coordenadas es diferente)
x_coords = {int(r["ID"]): float(r["Lat"]) for _, r in df_coords.iterrows()}
y_coords = {int(r["ID"]): float(r["Lon"]) for _, r in df_coords.iterrows()}

# --------------------------
# Depot
# Asumiendo que ID = 0 es el depósito, usamos sus coordenadas directamente
x_coords[0] = df_coords.loc[df_coords["ID"] == 0, "Lon"].values[0]
y_coords[0] = df_coords.loc[df_coords["ID"] == 0, "Lat"].values[0]

# Nodo ficticio para algoritmos de flujo (si aplica)
fictitious_node_id = max(x_coords.keys()) + 1
x_coords[fictitious_node_id] = x_coords[0]
y_coords[fictitious_node_id] = y_coords[0]

# --------------------------
# Leer KPI y flujos
df_kpi = pd.read_excel(file_kpi, engine="openpyxl")
df_flows = pd.read_excel(file_flows, engine="openpyxl")

# --------------------------
# NODOS REALES (sin depot ni ficticio)
nodes_real = [i for i in x_coords.keys() if i not in [0, fictitious_node_id]]

# --------------------------
# FIGURA INTERACTIVA
fig = go.Figure()
days = sorted(df_kpi["Day"].unique())

# --------------------------
# Loop por día para generar traces interactivos
for t in days:
    df_day_kpi = df_kpi[df_kpi["Day"] == t]
    df_day_flows = df_flows[df_flows["Day"] == t]

    # --------------------------
    # Nodos visitados y no visitados
    bins_visited = df_day_kpi["IDsBinsVisited"].values[0]
    bins_not_visited = df_day_kpi["IDsBinsNotVisited"].values[0]
    if isinstance(bins_visited, str):
        bins_visited = eval(bins_visited)
    if isinstance(bins_not_visited, str):
        bins_not_visited = eval(bins_not_visited)

    # --------------------------
    # Extraer KPIs de cada bin
    # Suponemos que df_day_kpi tiene columnas como:
    # 'S_current', 'Collected', 'Overflow', 'Profit', 'MG_member', 'Lookahead_fill'
    bin_kpis = {}
    for _, row in df_day_kpi.iterrows():
        for i in nodes_real:
            bin_kpis[i] = {
                "S_current": row.get(f"S_current_{i}", 0),
                "Collected": row.get(f"Collected_{i}", 0),
                "Overflow": row.get(f"Overflow_{i}", 0),
                "Profit": row.get(f"Profit_{i}", 0),
                "MG_member": row.get(f"MG_member_{i}", 0),
                "Lookahead_fill": row.get(f"Lookahead_fill_{i}", 0),
            }

    # --------------------------
    # TRACE NODOS VISITADOS con tooltip completo
    fig.add_trace(
        go.Scatter(
            x=[x_coords[i] for i in bins_visited],
            y=[y_coords[i] for i in bins_visited],
            mode="markers",
            marker=dict(color="green", size=10, symbol="circle"),
            name="Visited",
            text=[
                (
                    f"ID: {i}<br>"
                    f"Fill: {bin_kpis[i]['S_current']:.1f} kg<br>"
                    f"Collected: {bin_kpis[i]['Collected']:.1f} kg<br>"
                    f"Overflow: {bin_kpis[i]['Overflow']:.1f} kg<br>"
                    f"Profit: {bin_kpis[i]['Profit']:.2f}<br>"
                    f"MG member: {bin_kpis[i]['MG_member']}<br>"
                    f"Predicted Fill: {bin_kpis[i]['Lookahead_fill']:.1f} kg"
                )
                for i in bins_visited
            ],
            hoverinfo="text",
            visible=(t == days[0]),
        )
    )

    # TRACE NODOS NO VISITADOS
    fig.add_trace(
        go.Scatter(
            x=[x_coords[i] for i in bins_not_visited],
            y=[y_coords[i] for i in bins_not_visited],
            mode="markers",
            marker=dict(color="red", size=10, symbol="x"),
            name="Not Visited",
            text=[
                (
                    f"ID: {i}<br>"
                    f"Fill: {bin_kpis[i]['S_current']:.1f} kg<br>"
                    f"Collected: {bin_kpis[i]['Collected']:.1f} kg<br>"
                    f"Overflow: {bin_kpis[i]['Overflow']:.1f} kg<br>"
                    f"Profit: {bin_kpis[i]['Profit']:.2f}<br>"
                    f"MG member: {bin_kpis[i]['MG_member']}<br>"
                    f"Predicted Fill: {bin_kpis[i]['Lookahead_fill']:.1f} kg"
                )
                for i in bins_not_visited
            ],
            hoverinfo="text",
            visible=(t == days[0]),
        )
    )

    # DEPOT
    fig.add_trace(
        go.Scatter(
            x=[x_coords[0]],
            y=[y_coords[0]],
            mode="markers",
            marker=dict(color="blue", size=12, symbol="square"),
            name="Depot",
            text=["Depot"],
            hoverinfo="text",
            visible=(t == days[0]),
        )
    )

    # --------------------------
    # RUTAS (arcos activos)
    arcs = df_day_flows[df_day_flows["Var"] == "x"]
    for _, row in arcs.iterrows():
        i, j = int(row["i"]), int(row["j"])
        fig.add_trace(
            go.Scatter(
                x=[x_coords[i], x_coords[j]],
                y=[y_coords[i], y_coords[j]],
                mode="lines",
                line=dict(width=2, color="orange"),
                name="Route",
                visible=(t == days[0]),
                hoverinfo="none",
            )
        )

# --------------------------
# SLIDER POR DÍA
steps = []
for i, t in enumerate(days):
    step = dict(
        method="update",
        args=[
            {"visible": [False] * len(fig.data)},
            {"title": f"Day {t} Routes and Bins"},
        ],
        label=f"Day {t}",
    )
    n_traces_per_day = len(fig.data) // len(days)
    for j in range(n_traces_per_day):
        step["args"][0]["visible"][i * n_traces_per_day + j] = True
    steps.append(step)

sliders = [dict(active=0, currentvalue={"prefix": "Day: "}, pad={"t": 50}, steps=steps)]

# --------------------------
# LAYOUT FINAL
fig.update_layout(
    sliders=sliders,
    title="Dynamic VRPP Visualization - Full KPI Dashboard",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    showlegend=True,
    height=700,
)

# --------------------------
# MOSTRAR FIGURA
fig.show()

KeyError: 'Day'